In [1]:
from pyspark import SparkConf, SparkContext
import sys
import numpy as np

n = 1000
beta = 0.8
iterations = 40

In [2]:
def generateTuple(x):
        src, dst = x.split()
        return (int(src)-1, int(dst)-1)

def generateM(x):
    src, dist_list = x
    deg = len(dist_list)
    return [(dist, src, 1./deg) for dist in dist_list]

def generateR(x):
    r = np.zeros(n)
    for i,v in x:
        r[i] = v
    return r
conf = SparkConf()
sc = SparkContext(conf=conf)
data = sc.textFile("graph.txt")
data = data.map(lambda x: generateTuple(x)).distinct()# (s, d)
temp = data.groupByKey()# (s, resultiterable d_list)

M = temp.flatMap(lambda x: generateM(x))# (d, s, v)
r = np.ones(n) / n

for _ in range(iterations):
    # Broadcast the ranks dictionary to the workers
    broadcasted_ranks = sc.broadcast(dict(ranks.collect()))

    # Apply the generateM transformation using the broadcasted ranks
    M = links.flatMap(lambda x: generateM(x, broadcasted_ranks))

    # Proceed with the multiplication and update of ranks
    multiplications = M.map(lambda x: (x[0], x[2]))
    ranks = multiplications.reduceByKey(add).mapValues(lambda rank: beta * rank + (1 - beta) / n)

    
ascending = np.argsort(r.T) + 1
descending = np.argsort(-r.T) + 1

print('top 5 in descending order: ', descending[0:5])# [263 537 965 243 285]
print('bottom 5 in ascending order: ', ascending[0:5])# [558  93  62 424 408]


top 5 in descending order:  [263 537 965 243 285]
bottom 5 in ascending order:  [558  93  62 424 408]


In [3]:
sc.stop()


In [4]:

def generateL(x):
    src, dst = x.split()
    return (int(src)-1, int(dst)-1, 1)

def generateLT(x):
    src, dst = x.split()
    return (int(dst)-1, int(src)-1, 1)

def generateVector(x):
    r = np.zeros(n)
    for i,v in x:
        r[i] = v
    return r

conf = SparkConf()
sc = SparkContext(conf=conf)
data = sc.textFile("graph.txt")
L = data.map(lambda x: generateL(x)).distinct()
LT = data.map(lambda x: generateLT(x)).distinct()

a = np.ones(n)
for _ in range(iterations):
    muliplications = L.map(lambda f: (f[0], f[2]*a[f[1]]))
    h_rdd = muliplications.reduceByKey(lambda x, y : x + y)
    h = generateVector(h_rdd.collect())
    h = h / np.amax(h)

    muliplications = LT.map(lambda f: (f[0], f[2]*h[f[1]]))
    a_rdd = muliplications.reduceByKey(lambda x, y : x + y)
    a = generateVector(a_rdd.collect())
    a = a / np.amax(a)

h_ascending = np.argsort(h.T) + 1
h_descending = np.argsort(-h.T) + 1
print('top 5 hubs in descending order: ', h_descending[0:5])# [840 155 234 389 472]
print('bottom 5 hubs in ascending order: ', h_ascending[0:5])# [ 23 835 141 539 889]
a_ascending = np.argsort(a.T) + 1
a_descending = np.argsort(-a.T) + 1
print('top 5 authorities in descending order: ', a_descending[0:5])# [893  16 799 146 473]
print('bottom 5 authorities in ascending order: ', a_ascending[0:5])# [ 19 135 462  24 910]


top 5 hubs in descending order:  [840 155 234 389 472]
bottom 5 hubs in ascending order:  [ 23 835 141 539 889]
top 5 authorities in descending order:  [893  16 799 146 473]
bottom 5 authorities in ascending order:  [ 19 135 462  24 910]


In [5]:
sc.stop()